# Creando el modelo unificado de ruido

## 1.   Abstract

En esre notebook, haremos un analisis de diferentes simulaciones de ruido con tres canales diferentes:

* Canal de depolarización
* Canal SPAM
* Canal de relajación y desfase

La primera sección del notebook sera para definir los imports y demas funciones y datos globales. Luego, se hara una ejecución del algoritmo de Grover en el hardware de IBMQ $\textit{ibm_lagos}$, cuya topología es de 7 qubits. Este hardware sera usado no solo para una ejecución real de los circuitos, sino para tomar sus datos de calibración y usarlos a lo largo de los experimentos.

En la tercera sección, se haran multiples simulaciones del algoritmo de Grover, usando el simulador AER. Ademas, cada simulación se hara aplicandole un canal de de depolarización, con una probabilidad de ruido que incrementa del $0\%$ al $30\%$ y un step de $1\%$. Las simulaciones del algoritmo de Grover con el canal de depolarización se haran en tres experimentos diferentes:

    - La primera vez, para agregar ruido unicamente a compuertas de 1 qubit.
    - La segunda vez, para agregar ruido unicamente a compuertas de 2 qubits.
    - La tercera vez, para agregar ruido a ambos tipos de puerta.

En la cuarta sección, se haran simulaciones de un circuito al que se le aplicara un canal de relajación y desfase. El circuito sera llamado $iterative-bell$, que consiste en $n$ iteraciones de la aplicación de compuertas $h(q0),cx(q0,q1)$. Para cada simulación, el numero incrementara. Los datos usados para este canal seran los datos de calibración de $ibm\_lagos$.

En la quinta sección se haran simulaciones sobre el canal SPAM.

En la sexta sección, se probara una implementación de un $\textbf{Modelo Unificado de Ruido}$ (notado UNM) que implemente los tres canales en conjunto. Se simularan diferentes circuitos con cada uno de los canales del UNM por separado, y luego todos en conjunto. El objetivo de esta sección es validar el correcto funcionamiento del UNM y probar la simulación de ruido cuántico de los tres canales en simultaneo. 

Finalmente, se hara un analisis de los resultados obtenidos de todos los experimentos.

## 2.   Imports, funciones y datos globales

In [1]:
import numpy as np
from qiskit import QuantumCircuit,QuantumRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import GroverOperator, MCMT, ZGate,XGate,StatePreparation
import math
import matplotlib.pyplot as plt

# Importar desde el modulo de ruido Qiskit Aer
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

# Importar para correr en hardware de IBM Quantum
from  qiskit_ibm_provider import IBMProvider

# Importa el UNM
from unified_noise_model.Unified_Noise_Model import Unified_Noise_Model
from qiskit.circuit.library import XGate

ModuleNotFoundError: No module named 'unified_noise_model'

In [22]:
def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
    return qc

In [23]:
#---VARIABLES GLOBALES PARA CIRCUITO DE GROVER---
marked_states=['010']
nroQubits=3
shots=10000
errorProbabilities = np.arange(0,0.31,0.01)

#Se crea el operador de grover: oraculo + expansion de estado
oracle = grover_oracle(marked_states)
groverOp = GroverOperator(oracle)

#Se calcula el numero optimo de iteraciones
optimal_num_iterations = math.floor(
    math.pi / 4 * math.sqrt(2**groverOp.num_qubits / len(marked_states))
)

In [24]:
#crea ruido de depolarizacion
def add_depolarizing_error(noise_model,prob):
    noise_model = add_one_qubit_depolarizing_error(noise_model,prob)
    noise_model = add_two_qubits_depolarizing_error(noise_model,prob)
    return noise_model

#crea ruido de depolarizacion en compuertas 1 qubit
def add_one_qubit_depolarizing_error(noise_model,prob):
    error = depolarizing_error(prob, 1)
    noise_model.add_all_qubit_quantum_error(error,one_qubit_gates)
    return noise_model

#crea ruido de depolarizacion en compuertas 2 qubits 
def add_two_qubits_depolarizing_error(noise_model,prob):
    error = depolarizing_error(prob, 2)
    noise_model.add_all_qubit_quantum_error(error, two_qubits_gates)
    return noise_model

#PARA TESTING
#noise_model = NoiseModel()
#print(add_one_qubit_depolarizing_error(noise_model,0.1))
#print(add_two_qubits_depolarizing_error(noise_model,0.3))
#print(add_depolarizing_error(noise_model,0.2))

In [25]:
def add_relaxation_dephasing_channel(qubits,qubitsT1,qubitsT2,singleGates,singleGatesTimes,
                                     doubleGates=[],doubleGatesTime=0):

    noise_model = NoiseModel()

    #Para cada tupla (qubit,compuerta), se agrega error al modelo con el tiempo de la respectiva gate, y
    #y los T1,T2 del respectivo qubit.
    
    for j in qubits:
        for i in range(len(singleGates)):
            error = thermal_relaxation_error(qubitsT1[j],qubitsT2[j],singleGatesTimes[j][i])
            noise_model.add_quantum_error(error,singleGates[i],[j])

        for j2 in qubits:
            for i in range(len(doubleGates)):
                error = thermal_relaxation_error(qubitsT1[j],qubitsT2[j],doubleGatesTime[i]).expand(
                thermal_relaxation_error(qubitsT1[j2],qubitsT2[j2],doubleGatesTime[i]))
                noise_model.add_quantum_error(error,doubleGates[i],[j,j2])
        
    return noise_model
  
#PARA TESTING
#noise_model = add_relaxation_dephasing_channel([0,1],[2,3],[1,2],['x','z'],[[4,2],[3,2]],['cz','cx'],[2,3])
#print(noise_model)

In [26]:
def add_spam_channel(statePreparation_error_prob=-1,measurement_error_prob=-1,statePreparation_error_gate=None):
    '''
    Recibe por parametros la probabilidad de error de medicion, de preparacion de esstado y la compuerta que representa
    el error de preparacion de estado en el circuito. Esta compuerta debe ser una XGate() con un label asignado.
    '''
    noise_model = NoiseModel()
    
    if(statePreparation_error_prob >=0):
        #print(statePreparation_error_gate.label)
        statePreparation_error = pauli_error([('I',statePreparation_error_prob),('X',1-statePreparation_error_prob)])
        noise_model.add_all_qubit_quantum_error(statePreparation_error,statePreparation_error_gate.label)
        noise_model.add_basis_gates(['x'])

    
    if(measurement_error_prob >=0):
        measurement_error = pauli_error([('X',measurement_error_prob),('I',1-measurement_error_prob)])
        noise_model.add_all_qubit_quantum_error(measurement_error,"measure")
    
    print(noise_model)
    return noise_model
 
#TESTING
#add_spam_channel(0.2,0.5,XGate(label='prueba'))

In [27]:
token='825a8b2e6fb861f63088f866e59357e49447b4b9b24ff6136e56dcc28f2fb0cedda6c2976b9e2aef48129a80864265494be7f8a7aae5b9ebd6654a7d80d9b5cb'

In [28]:
IBMProvider.save_account(token= token, overwrite=True)
provider = IBMProvider()
print(provider.backends())

[<IBMBackend('ibmq_qasm_simulator')>, <IBMBackend('simulator_extended_stabilizer')>, <IBMBackend('simulator_mps')>, <IBMBackend('simulator_statevector')>, <IBMBackend('simulator_stabilizer')>, <IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyoto')>]


In [29]:
backend= provider.get_backend('ibm_lagos')
conf = backend.configuration()
props = backend.properties()

QiskitBackendNotFoundError: 'No backend matches the criteria'

In [11]:
#---OBTENER PARAMETROS DEL BACKEND---

qubits = range(conf.n_qubits)
t1=[]
t2=[]
#Se ven las basis gates del backend para despues filtrarlas segun si son single o double
basis_gates = conf.basis_gates
print('basis gates:',basis_gates,'\n')
#Se toman las single gates
one_qubit_gates = conf.basis_gates[2:5]
one_qubit_gates_times=[]
#Se toman las double gates
two_qubits_gates = conf.basis_gates[0:1]
#VALOR FIXED TEMPORAL,VER DESPUES CON VICTOR
#TEMA TOPOLOGICO DEL HARDWARE
two_qubits_gates_times=[props.gate_length('cx',[0,1])]

#para cada qubit se obtiene:
    #su T1
    #su T2
    # para cada single gate, se obtiene el tiempo de gate.
for i in qubits:
    t1.append(props.t1(i))
    t2.append(props.t2(i))
    
    current_qubit_gates_times = []
    for j in one_qubit_gates:
        current_qubit_gates_times.append(props.gate_length(j,i))
    one_qubit_gates_times.append(current_qubit_gates_times)

#Se imprime por pantalla los datos obtenidos
print('t1:',t1,'\n')
print('t2:',t2,'\n')
print()
print('single basis gates:',one_qubit_gates,'\n')
print('single basis gates times:',one_qubit_gates_times,'\n')
print('double basis gates:',two_qubits_gates,'\n')
print('double basis gates times:',two_qubits_gates_times,'\n')

print('------------------------------------------------------------------------------------------------------------------------')

#---CREAR MODELO DE RUIDO---
noise_model = add_relaxation_dephasing_channel(qubits,t1,t2,one_qubit_gates,one_qubit_gates_times,two_qubits_gates,
                                               two_qubits_gates_times)
print(noise_model)

## 3.   Ejecución del Algoritmo de Grover en hardware real - ibm_lagos

Esta ejecución se realizo con los siguientes parametros:

* $010$ como estado marcado
* circuito de tres qubits
* 10000 shots del circuito.


In [ ]:
from PIL import Image
img = Image.open(r'C:\Users\54299\Desktop\Facultad\Trabajos\QuantumQuipu\Noise and native gates\QuantumQuipu_NoiseAndNativeGates\Imagenes\ibm_lagos_result.png')
display(img)

## 4.   Simulaciones del canal de depolarización

El canal de depolarización se probara sobre un algoritmo de Grover con los siguientes parametros:
* $010$ como estado marcado
* circuito de tres qubits
* 10000 shots del circuito.

En cada experimento, se ira aumentado la probabilidad de error de las simulaciones desde $0\%$ al $30\%$ con un step de $1\%$. Ademas, se mostraran sus respectivos gráficos con los resultados y un contador que muestra la cantidad de compuertas ruidosas del algoritmo ejecutado.

### 4.1   Errores de depolarizacion en compuertas de 1 qubit

In [12]:
results = []
for i in errorProbabilities:
    #ALGORITMO
    print('PROBABILIDAD DE ERROR: ',i)
     
    noiseModel = add_one_qubit_depolarizing_error(NoiseModel(),i)
    print('noise model: ',noiseModel)
        
    qr = QuantumRegister(nroQubits)
    qc = QuantumCircuit(qr)
    qc.h(qr)
    qc.compose(groverOp.decompose(), inplace=True)
    qc.measure_all()
    #display(qc.draw("mpl"))
    #display(qc.data)
    
    #EJECUCION
    sim_noise = AerSimulator(noise_model=noiseModel)
    circ_tnoise = transpile(qc, sim_noise,basis_gates=basis_gates)
    display(circ_tnoise.draw('mpl'))
    result_bit_flip = sim_noise.run(circ_tnoise,shots=shots).result()
    counts_bit_flip = result_bit_flip.get_counts(0)
    display(plot_histogram(counts_bit_flip))
        
    results.append(counts_bit_flip)

In [13]:
#CALCULO DE PROBABILIDADES DE EXITO PARA CADA EXPERIMENTO
successCounts1=[]
for i in results:
    currentCount = 0
    for j in marked_states:
        currentCount = currentCount + i[j]
    successCounts1.append(currentCount/shots)
#print(successCounts1)

In [14]:
# Crear el gráfico
plt.plot(errorProbabilities, successCounts1, marker='o', linestyle='-')

# Etiquetas de los ejes
plt.xlabel('Probabilidad de error')
plt.ylabel('Probabilidad de exito')

# Título del gráfico
plt.title('Probabilidad de exito con errores de Pauli en compuertas de 1 qubit')

# Mostrar el gráfico
plt.grid(True)
plt.show()

In [ ]:
#CALCULO DE CANTIDAD DE COMPUERTAS UTILIZADAS
#display(circ_tnoise.count_ops())
ops = circ_tnoise.count_ops()
single_qubit_ops = ops['rz']+ops['sx']
print('-----------------------------------------------------------')
print('Cantidad de compuertas de 1 qubit: ',single_qubit_ops)
print('-----------------------------------------------------------')

### 4.2   Errores de depolarizacion en compuertas de 2 qubits

In [15]:
results = []
for i in errorProbabilities:
    #ALGORITMO
    print('PROBABILIDAD DE ERROR: ',i)
    
    noiseModel = add_two_qubits_depolarizing_error(NoiseModel(),i)
    basisGates=noiseModel.basis_gates
    print('noise model: ',noiseModel)
        
    qr = QuantumRegister(nroQubits)
    qc = QuantumCircuit(qr)
    qc.h(qr)
    qc.compose(groverOp.decompose(), inplace=True)
    qc.measure_all()
    #display(qc.draw("mpl"))
    #display(qc.data)
    
    #EJECUCION
    sim_noise = AerSimulator(noise_model=noiseModel)
    circ_tnoise = transpile(qc, sim_noise,basis_gates=basis_gates)
    display(circ_tnoise.draw('mpl'))
    result_bit_flip = sim_noise.run(circ_tnoise,shots=shots).result()
    counts_bit_flip = result_bit_flip.get_counts(0)
    display(plot_histogram(counts_bit_flip))
        
    results.append(counts_bit_flip)

In [16]:
#CALCULO DE PROBABILIDADES DE EXITO PARA CADA EXPERIMENTO
successCounts2=[]
for i in results:
    currentCount = 0
    for j in marked_states:
        currentCount = currentCount + i[j]
    successCounts2.append(currentCount/shots)
#print(successCounts2)

In [17]:
# Crear el gráfico
plt.plot(errorProbabilities, successCounts2, marker='o', linestyle='-')

# Etiquetas de los ejes
plt.xlabel('Probabilidad de error')
plt.ylabel('Probabilidad de exito')

# Título del gráfico
plt.title('Probabilidad de exito con errores de Pauli en compuertas de 2 qubits')

# Mostrar el gráfico
plt.grid(True)
plt.show()

In [18]:
#CALCULO DE CANTIDAD DE COMPUERTAS UTILIZADAS
#display(circ_tnoise.count_ops())
ops = circ_tnoise.count_ops()
two_qubits_ops = ops['cx']
print('-----------------------------------------------------------')
print('Cantidad de compuertas de 2 qubit: ',two_qubits_ops)
print('-----------------------------------------------------------')

### 4.3 Errores de depolarizacion en todas las compuertas

In [19]:
results = []
for i in errorProbabilities:
    #ALGORITMO
    print('PROBABILIDAD DE ERROR: ',i)
    
    noiseModel = add_depolarizing_error(NoiseModel(),i)
    basisGates=noiseModel.basis_gates
    print('noise model: ',noiseModel)
        
    qr = QuantumRegister(nroQubits)
    qc = QuantumCircuit(qr)
    qc.h(qr)
    qc.compose(groverOp.decompose(), inplace=True)
    qc.measure_all()
    #display(qc.draw("mpl"))
    #display(qc.data)
    
    #EJECUCION
    sim_noise = AerSimulator(noise_model=noiseModel)
    circ_tnoise = transpile(qc, sim_noise,basis_gates=basis_gates)
    display(circ_tnoise.draw('mpl'))
    result_bit_flip = sim_noise.run(circ_tnoise,shots=shots).result()
    counts_bit_flip = result_bit_flip.get_counts(0)
    display(plot_histogram(counts_bit_flip))
        
    results.append(counts_bit_flip)

In [20]:
#CALCULO DE PROBABILIDADES DE EXITO PARA CADA EXPERIMENTO
successCounts3=[]
for i in results:
    currentCount = 0
    for j in marked_states:
        currentCount = currentCount + i[j]
    successCounts3.append(currentCount/shots)
#print(successCounts3)

In [ ]:
# Crear el gráfico
plt.plot(errorProbabilities[:16], successCounts3[:16], marker='o', linestyle='-')

# Etiquetas de los ejes
plt.xlabel('Probabilidad de error')
plt.ylabel('Probabilidad de exito')

# Título del gráfico
plt.title('Probabilidad de exito con errores de Pauli en todas las compuertas')

# Mostrar el gráfico
plt.grid(True)
plt.show()

In [ ]:
#CALCULO DE CANTIDAD DE COMPUERTAS UTILIZADAS
#display(circ_tnoise.count_ops())
ops = circ_tnoise.count_ops()
ops = ops['cx']+ops['rz']+ops['sx']
print('-----------------------------------------------------------')
print('Cantidad de compuertas de 2 qubit: ',ops)
print('-----------------------------------------------------------')

## 5. Errores del canal de  relajación y desfase

Para este canal, no tenemos un parametro de probabilidad de error como si lo tenemos para el canal de depolarización. Los parametros a modificar para ver como afecta este canal son:

1. Profundidad del circuito
2. Tiempo de ejecución de las compuertas, para cada qubit
3. t1 y t2, para cada qubit

Empezaremos solo probando modificar la profundidad del circuito utilizando un iterative-bell. Este algoritmo genera $n$ repeticiones de una compuerta $H$ y una compuerta $CX$.En un futuro, haremos pruebas mas complejas con el resto de parametros.

In [ ]:
#AGREGAR AL CIRCUITO UN SWAP TEST PARA VER LA SIMILITUD
results = []
for n in range(0,51):
    #ALGORITMO
    noise_model = add_relaxation_dephasing_channel(qubits,t1,t2,one_qubit_gates,one_qubit_gates_times,two_qubits_gates,
                                                   two_qubits_gates_times)

    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.cx(0,1)
    for i in range(n):
        qc.x(0)
        qc.x(0)
        qc.x(1)
        qc.x(1)
    qc.measure(range(2),range(2))
    
    
    #EJECUCION
    simulator = AerSimulator()
    noisy_simulator = AerSimulator(noise_model=noise_model)

    qc_transpiled = transpile(qc,simulator,basis_gates=conf.basis_gates,optimization_level=0)
    noisy_qc_transpiled = transpile(qc,noisy_simulator,basis_gates=basis_gates,optimization_level=0)
    print('N = ',n)
    print('Iterative-bell descompuesto en compuertas nativas:')
    display(qc_transpiled.draw('mpl'))

    result = simulator.run(qc_transpiled,shots=shots).result()
    noisy_result = noisy_simulator.run(noisy_qc_transpiled,shots=shots).result()

    counts = result.get_counts(0)
    noisy_counts = noisy_result.get_counts(0)

    #print('Resultados ideales:')
    #display(plot_histogram(counts))
    
    print('Resultados con ruido:')
    display(plot_histogram(noisy_counts))
    
    results.append(noisy_counts)   

    print('-----------------------------------------------------------------------------------------------------------------------------')

In [ ]:
#CALCULO DE PROBABILIDADES DE EXITO PARA CADA SIMULACION
successCounts4=[]
for i in results:
    successCounts4.append((i['00']+i['11'])/shots)
#print(successCounts4)

In [ ]:
# Crear el gráfico
plt.plot(range(0,51), successCounts4, marker='o', linestyle='-')

# Etiquetas de los ejes
plt.xlabel('Cantidad de iteraciones de pares de compuertas X')
plt.ylabel('Probabilidad de exito')

# Título del gráfico
plt.title('Probabilidad de exito con canal de relajación y desfase')

# Mostrar el gráfico
plt.grid(True)
plt.show()

## 6. Canal de SPAM

IBM Quantum proporciona información sobre el $\textit{Prob meas 1 prep 0}$ y $\textit{Prob meas 0 prep 1}$ para sus computadoras cuánticos. Estos datos representan, para cada qubit, la probabilidad de medir $1$ exactamente despues de haber preparado el qubit en $0$, y viceversa. Hasta ahora, hemos implementado este canal separando la probabilidad de medición y la de preparación de estado. Se recibe como parametro un $p_2$ que indica la probabilidad de error en la medición (dato encontrado en IBM Quantum como $\textit{Readout Assignment error}$) y un $p'_2$ que indica la probabilidad de error en la preparación de estado (dato que, creemos , son los primeros dos mencionados arriba). Para implementar el ruido de medición, usamos errores de pauli sobre la operación $measurement$, pero para el ruido de preparación de estado, agregamos un capa extra de compuertas $X$ entre la preparación de estado y el circuito como tal.Sobre esta capa de $X$ se aplica la probabilidad de ruido. A futuro debemos plantearnos las siguientes dudas:

* ¿Que representa, realmente, $\textit{Prob meas 1 prep 0}$ y $\textit{Prob meas 0 prep 1}$?
* ¿Sera necesario realizar modificacion sobre nuestro canal SPAM para incorporar estos dos parametros?

Estas preguntas vienen en base que, si bien creemos que los parametros representan el ruido de preparación de estado,hemos leido por internet que otras personas lo consideran una combinación del ruido de preparación de estado junto al de medición. Si resulta ser esto, probablemente hara falta modificar la implementación de este canal.

### 6.1 Errores de medición

In [ ]:
results = []
for i in errorProbabilities:
    #ALGORITMO
    print('PROBABILIDAD DE ERROR: ',i)

    noiseModel = add_spam_channel(measurement_error_prob=i)
    print('noise model: ',noiseModel)
        
    qr = QuantumRegister(nroQubits)
    qc = QuantumCircuit(qr)
    qc.h(qr)   
    qc.compose(groverOp.decompose(), inplace=True)
    qc.measure_all()
    #display(qc.draw("mpl"))
    #display(qc.data)
    
    #EJECUCION
    sim_noise = AerSimulator(noise_model=noiseModel)
    circ_tnoise = transpile(qc, sim_noise,basis_gates=basis_gates)
    display(circ_tnoise.draw('mpl'))
    result = sim_noise.run(circ_tnoise,shots=shots).result()
    counts = result.get_counts(0)
    display(plot_histogram(counts))
        
    results.append(counts)

In [ ]:
#CALCULO DE PROBABILIDADES DE EXITO PARA CADA EXPERIMENTO
successCounts5=[]
for i in results:
    currentCount = 0
    for j in marked_states:
        currentCount = currentCount + i[j]
    successCounts5.append(currentCount/shots)
print(successCounts5)

In [ ]:
# Crear el gráfico
plt.plot(errorProbabilities, successCounts5, marker='o', linestyle='-')

# Etiquetas de los ejes
plt.xlabel('Probabilidad de error')
plt.ylabel('Probabilidad de exito')

# Título del gráfico
plt.title('Probabilidad de exito con errores de medición')

# Mostrar el gráfico
plt.grid(True)
plt.show()

### 6.2 Errores de preparación de estado

In [ ]:
results = []
for i in errorProbabilities:
    #ALGORITMO
    print('PROBABILIDAD DE ERROR: ',i)
     
    # Compuerta X que se usara para representar el error de preparacion de estado
    x_StatePreparation = XGate(label='x_StatePreparation')
        
    noiseModel = add_spam_channel(statePreparation_error_prob= i,statePreparation_error_gate= x_StatePreparation)
    print('noise model: ',noiseModel)
        
    qr = QuantumRegister(nroQubits)
    qc = QuantumCircuit(qr)
    qc.h(qr)
        
    # Debemos crear manualmente una barrera de compuertas X que representaran el error de preparacion de estado
    for q in range(nroQubits):
        qc.append(x_StatePreparation,[q])
    
    qc.compose(groverOp.decompose(), inplace=True)
    qc.measure_all()
    #display(qc.draw("mpl"))
    #display(qc.data)
    
    #EJECUCION
    simulator = AerSimulator(noise_model=noiseModel)
    circ_tnoise = transpile(qc, simulator,basis_gates=basis_gates,optimization_level=0)
    display(circ_tnoise.draw('mpl'))
    result = simulator.run(circ_tnoise,shots=shots).result()
    counts = result.get_counts(0)
    display(plot_histogram(counts))
        
    results.append(counts)

In [ ]:
#CALCULO DE PROBABILIDADES DE EXITO PARA CADA EXPERIMENTO
successCounts6=[]
for i in results:
    currentCount = 0
    for j in marked_states:
        currentCount = currentCount + i[j]
    successCounts6.append(currentCount/shots)
#print(successCounts6)

In [ ]:
# Crear el gráfico
plt.plot(errorProbabilities, successCounts6, marker='o', linestyle='-')

# Etiquetas de los ejes
plt.xlabel('Probabilidad de error')
plt.ylabel('Probabilidad de exito')

# Título del gráfico
plt.title('Probabilidad de exito con errores de preparación de estado')

# Mostrar el gráfico
plt.grid(True)
plt.show()

Se puede observar que, a diferencia del ruido de medición, el ruido de preparación de estado no aparece en ningún momento. La razón de esto es que este ruido se representa como una probabilidad $p'_2$ de que se aplique una compuerta $X$ luego de la preparación de estado. En el caso de grover's search, como la preparación de estado deja un registro de $n$ qubits en el estado $|+\rangle$, la aplicación de un error representado por una compuerta $X$ no tiene efecto alguno.

Para probar el correcto funcionamiento de este tipo de ruido, probaremos el siguiente circuito simple:

In [ ]:
results=[]
for i in errorProbabilities: 
    x_StatePreparation = XGate(label='x_StatePreparation')

    noise_model = add_spam_channel(i,0,x_StatePreparation)
    
    qc = QuantumCircuit(2,2)
    a = StatePreparation([0,1])
    qc.append(a,[0])
    for q in range(2):
        qc.append(x_StatePreparation,[q])
    qc.barrier()
    qc.measure(range(2),range(2))


    simulator = AerSimulator(
        noise_model=noise_model
    )
    qc_transpiled = transpile(qc,simulator,optimization_level=0)
    display(qc_transpiled.draw('mpl'))
    result = simulator.run(qc_transpiled,shots=shots).result()
    counts = result.get_counts(0)
    display(plot_histogram(counts))
    
    results.append(counts)

In [ ]:
#CALCULO DE PROBABILIDADES DE EXITO PARA CADA EXPERIMENTO
successCounts7=[]
for i in results:
    successCounts7.append(i['01']/shots)
print(successCounts7)

In [ ]:
# Crear el gráfico
plt.plot(errorProbabilities, successCounts7, marker='o', linestyle='-')

# Etiquetas de los ejes
plt.xlabel('Probabilidad de error')
plt.ylabel('Probabilidad de exito')

# Título del gráfico
plt.title('Probabilidad de exito con errores de preparación de estado (NO EN GROVER\'S SEARCH)')

# Mostrar el gráfico
plt.grid(True)
plt.show()

## Conclusiones

Los resultados obtenidos para simulación de ruido de depolarizacion en compuertas de 1 qubit y de 2 qubits son los siguientes: 

In [ ]:
# Crear el gráfico
plt.plot(errorProbabilities, successCounts1, marker='o', linestyle='-', label='Ruido en compuertas 1Q')
plt.plot(errorProbabilities, successCounts2, marker='o', linestyle='-', label='Ruido en compuertas 2Q')
plt.plot(errorProbabilities, successCounts3, marker='o', linestyle='-', label='Ruido en todas las compuertas')


# Etiquetas de los ejes
plt.xlabel('Probabilidad de error')
plt.ylabel('Probabilidad de exito')

# Título del gráfico
plt.title('Probabilidad de exito con errores de depolarizacion')

plt.legend()

# Mostrar el gráfico
plt.grid(True)
plt.show()

Se sabe que, desde un punto teorico, el ruido generado en compuerta de dos qubits afecta mas al resultado que el ruido en compuertas de un qubit. Este hecho no se ve representado en la grafica previamente mostrada. Creemos que esto se debe, principalmente,a la diferencia en cantidad de compuertas de 1 qubit respecto a las de 2 qubits.

## Referencias

1. StackExchange - Quantum Computing | How to actually send Qiskit code to the que to be ran on an IBM Quantum Computer? https://quantumcomputing.stackexchange.com/questions/26863/how-to-actually-send-qiskit-code-to-the-que-to-be-ran-on-an-ibm-quantum-computer 

2. How to Show Images in Jupyter Notebook?
https://saturncloud.io/blog/how-to-show-images-in-jupyter-notebook/

3. Qiskit - Building noise models
https://qiskit.org/ecosystem/aer/tutorials/3_building_noise_models.html

4. IBM Quantum calibration data
https://quantumcomputing.stackexchange.com/questions/17054/characteristics-of-the-ibm-quantum-computer